# Cradle Data Load Configuration - Interactive UI Widget Example

This notebook demonstrates how to use the Cradle Config Widget to replace manual configuration blocks with an interactive UI.

## 🎯 What This Notebook Does

1. Sets up a basic pipeline configuration
2. Creates an interactive UI widget for Cradle data loading configuration
3. Generates a `CradleDataLoadConfig` object that can be used in your pipeline
4. Shows how to add the config to your `config_list`

## 📋 Prerequisites

- The Cradle UI server should be running on port 8001
- Required packages: `ipywidgets`, `requests`

## Step 1: Setup and Imports

In [ ]:
# Standard imports
import sys
from pathlib import Path
import json
from datetime import datetime

# Add project root to path
project_root = str(Path().absolute().parent.parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to path: {project_root}")

print("✅ Imports and path setup complete")

## Step 2: Create Base Configuration

First, let's create a basic pipeline configuration that the widget will use as a foundation.

In [ ]:
# Import base configuration
from cursus.core.base.config_base import BasePipelineConfig

# Create a sample base configuration
base_config = BasePipelineConfig(
    bucket="my-pipeline-bucket",
    current_date="2025-10-06",
    region="NA",
    aws_region="us-east-1",
    author="demo-user",
    role="arn:aws:iam::123456789012:role/demo-role",
    service_name="DemoService",
    pipeline_version="1.0.0",
    project_root_folder="demo-project",  # Added missing required field
    framework_version="2.1.0",
    py_version="py310",
    source_dir="/path/to/source"
)

print("✅ Base configuration created:")
print(f"   Author: {base_config.author}")
print(f"   Region: {base_config.region}")
print(f"   Service: {base_config.service_name}")
print(f"   Bucket: {base_config.bucket}")
print(f"   Project Root: {base_config.project_root_folder}")

## Step 3: Initialize Config List

Create the config list that will store all our pipeline configurations.

In [ ]:
# Initialize config list (same as in demo_config.ipynb)
config_list = []

# Add base config to the list
config_list.append(base_config)

print(f"✅ Config list initialized with {len(config_list)} item(s)")

## Step 4: Check Server Status

Let's verify that the Cradle UI server is running.

In [ ]:
import requests

def check_server_status(port=8001):
    """Check if the Cradle UI server is running."""
    try:
        response = requests.get(f"http://localhost:{port}/health", timeout=5)
        if response.status_code == 200:
            print(f"✅ Server is running on port {port}")
            return True
        else:
            print(f"❌ Server responded with status {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ Server is not running on port {port}")
        print(f"   Error: {str(e)}")
        print(f"   Please start the server with:")
        print(f"   cd src/cursus/api/cradle_ui && uvicorn app:app --host 0.0.0.0 --port {port} --reload")
        return False

server_running = check_server_status()

if not server_running:
    print("\n⚠️ Please start the server before proceeding to the next step.")

## Step 5: Create Training Configuration Widget

Now let's create the interactive widget for training data configuration. This replaces the complex manual configuration block.

In [ ]:
# Import the widget
from cursus.api.cradle_ui.jupyter_widget import create_cradle_config_widget

# Create the training configuration widget
print("🎯 Creating Training Data Configuration Widget")
print("=" * 50)

training_widget = create_cradle_config_widget(
    base_config=base_config,
    job_type="training",
    height="700px"  # Adjust height as needed
)

# Display the widget
training_widget.display()

print("\n📝 Instructions:")
print("1. Complete the 4-step configuration in the UI above")
print("2. Click 'Finish' in the UI to generate the configuration")
print("3. Run the next cell to retrieve the configuration object")

## Step 6: Get Training Configuration

After completing the UI configuration above, run this cell to get the generated config object.

In [ ]:
# Get the training configuration using the new JSON file workflow
# After completing the UI above:
# 1. Click 'Finish' in the UI to generate the configuration
# 2. Click 'Get Configuration' button below to save to JSON file
# 3. Load the saved configuration using the code below

from cursus.api.cradle_ui.utils.config_loader import load_cradle_config_from_json

# First, click the 'Get Configuration' button above to save the config to a JSON file
# Then update the file path below to match where you saved it

# Example: Load configuration from JSON file
try:
    # Update this path to match where you saved your configuration
    config_file_path = 'cradle_config_training.json'  # Update this path!
    
    # Load the configuration (this properly handles all nested objects)
    training_cradle_data_load_config = load_cradle_config_from_json(config_file_path)
    
    print("✅ Training configuration loaded successfully!")
    print("=" * 50)
    print(f"Job Type: {training_cradle_data_load_config.job_type}")
    print(f"Author: {training_cradle_data_load_config.author}")
    print(f"Region: {training_cradle_data_load_config.region}")
    print(f"Data Sources: {len(training_cradle_data_load_config.data_sources_spec.data_sources)}")
    print(f"Start Date: {training_cradle_data_load_config.data_sources_spec.start_date}")
    print(f"End Date: {training_cradle_data_load_config.data_sources_spec.end_date}")
    print(f"Output Format: {training_cradle_data_load_config.output_spec.output_format}")
    print(f"Cluster Type: {training_cradle_data_load_config.cradle_job_spec.cluster_type}")
    
    # Add to config list (same as manual configuration)
    config_list.append(training_cradle_data_load_config)
    print(f"\n✅ Added to config_list. Total configs: {len(config_list)}")
    
except FileNotFoundError:
    print("⚠️ Configuration file not found.")
    print("Please:")
    print("1. Complete the configuration in the UI above")
    print("2. Click 'Finish' in the UI")
    print("3. Click 'Get Configuration' button to save the JSON file")
    print("4. Update the config_file_path variable above with the correct path")
    
except Exception as e:
    print(f"❌ Error loading configuration: {str(e)}")
    print("Please ensure the JSON file was saved correctly from the UI.")

## Step 7: Create Calibration Configuration Widget (Optional)

Let's also create a calibration configuration to show how you can create multiple configs.

In [ ]:
# Create calibration configuration widget
print("🎯 Creating Calibration Data Configuration Widget")
print("=" * 50)

calibration_widget = create_cradle_config_widget(
    base_config=base_config,
    job_type="calibration",
    height="700px"
)

# Display the widget
calibration_widget.display()

print("\n📝 Instructions:")
print("1. Complete the 4-step configuration in the UI above")
print("2. Click 'Finish' in the UI to generate the configuration")
print("3. Run the next cell to retrieve the configuration object")

## Step 8: Get Calibration Configuration

In [ ]:
# Get the calibration configuration
calibration_cradle_data_load_config = calibration_widget.get_config()

if calibration_cradle_data_load_config:
    print("✅ Calibration configuration generated successfully!")
    print("=" * 50)
    print(f"Job Type: {calibration_cradle_data_load_config.job_type}")
    print(f"Author: {calibration_cradle_data_load_config.author}")
    print(f"Region: {calibration_cradle_data_load_config.region}")
    print(f"Data Sources: {len(calibration_cradle_data_load_config.data_sources_spec.data_sources)}")
    print(f"Start Date: {calibration_cradle_data_load_config.data_sources_spec.start_date}")
    print(f"End Date: {calibration_cradle_data_load_config.data_sources_spec.end_date}")
    print(f"Output Format: {calibration_cradle_data_load_config.output_spec.output_format}")
    print(f"Cluster Type: {calibration_cradle_data_load_config.cradle_job_spec.cluster_type}")
    
    # Add to config list
    config_list.append(calibration_cradle_data_load_config)
    print(f"\n✅ Added to config_list. Total configs: {len(config_list)}")
    
else:
    print("⚠️ No configuration available.")
    print("Please complete the configuration in the UI above and click 'Finish'.")

## Step 9: Verify and Display Final Results

Let's verify that our configurations were created correctly and show the final config list.

In [ ]:
# Display final results
print("🎉 Configuration Generation Complete!")
print("=" * 50)
print(f"Total configurations in config_list: {len(config_list)}")
print()

for i, config in enumerate(config_list):
    config_type = type(config).__name__
    print(f"{i+1}. {config_type}")
    
    if hasattr(config, 'job_type'):
        print(f"   Job Type: {config.job_type}")
    if hasattr(config, 'author'):
        print(f"   Author: {config.author}")
    if hasattr(config, 'region'):
        print(f"   Region: {config.region}")
    print()

print("✅ All configurations are ready for use in your pipeline!")

## Step 10: Export Configuration (Optional)

You can also export the configurations to JSON for later use.

In [ ]:
# Export configurations to JSON
def export_configs_to_json(config_list, filename="cradle_configs.json"):
    """Export configurations to JSON file."""
    try:
        # Convert configs to dictionaries
        config_dicts = []
        for config in config_list:
            if hasattr(config, 'model_dump'):
                config_dicts.append({
                    'type': type(config).__name__,
                    'data': config.model_dump()
                })
            else:
                config_dicts.append({
                    'type': type(config).__name__,
                    'data': str(config)
                })
        
        # Save to file
        with open(filename, 'w') as f:
            json.dump(config_dicts, f, indent=2, default=str)
        
        print(f"✅ Configurations exported to {filename}")
        return True
        
    except Exception as e:
        print(f"❌ Error exporting configurations: {str(e)}")
        return False

# Export the configurations
export_configs_to_json(config_list, "example_cradle_configs.json")

## 🎯 Summary

This notebook demonstrated how to:

1. ✅ **Set up the environment** and base configuration
2. ✅ **Create interactive UI widgets** for Cradle data loading configuration
3. ✅ **Generate valid CradleDataLoadConfig objects** through the UI
4. ✅ **Add configurations to config_list** (same as manual approach)
5. ✅ **Export configurations** for later use

## 🔄 Comparison with Manual Configuration

**Manual Approach (Complex):**
```python
training_cradle_data_load_config = create_cradle_data_load_config(
    base_config=base_config,
    job_type='training',
    mds_field_list=mds_field_list,
    start_date=training_start_datetime,
    end_date=training_end_datetime,
    # ... 15+ more parameters
)
```

**Widget Approach (Simple):**
```python
widget = create_cradle_config_widget(base_config=base_config, job_type="training")
widget.display()
training_cradle_data_load_config = widget.get_config()
```

## 🎉 Benefits

- **🎨 User-Friendly**: Visual interface instead of complex parameter lists
- **🛡️ Error Prevention**: Built-in validation and error checking
- **⚡ Faster**: Complete configuration in minutes, not hours
- **🔄 Same Output**: Identical CradleDataLoadConfig objects
- **📋 Easy Integration**: Works with existing notebook workflows

The widget provides the exact same functionality as manual configuration but with a dramatically improved user experience!